## solution of the LASSO problem
Data generation:

In [2]:
import numpy as np


In [ ]:
# Choose a random matrix 100x200 using iid Gaussian distribution with randn() function

def random_matrix():
    return np.random.randn(100, 200)

# choose a a sparse vector xˆ ∈ R200: uniformly choose 10% of the entries to be non-zeros, and randomly choose their values, again using randn().

def random_sparse_vector():
    x = np.zeros(200)
    non_zero_indices = np.random.choice(200, 20, replace=False)
    x[non_zero_indices] = np.random.randn(20)
    return x

A = random_matrix()
x = random_sparse_vector()

y = A @ x

#  normalize x so that the average value in magnitude of Ax is 1
x = x / (0.01*np.linalg.norm(y, ord=1))

# Add noise to the data
noise_std_dev = 0.01
eta = np.random.randn(100) * noise_std_dev

# Compute b = Ax + η
b = A @ x + eta


In [34]:
# Now we will try to reconstruct xˆ given A, b, by solving the LASSO problem using PCD or ISTA

# LASSO problem using PCD
def PCD(A, b, lmbda, num_iter=100):
    x = np.zeros(A.shape[1])
    for _ in range(num_iter):
        for i in range(A.shape[1]):
            x[i] = 0
            r = b - A @ x
            a = A[:, i]
            x[i] = np.dot(a, r) / np.dot(a, a)
            x[i] = np.sign(x[i]) * max(0, abs(x[i]) - lmbda)
    return x

# LASSO problem using ISTA
def ISTA(A, b, lmbda, num_iter=100):
    x = np.zeros(A.shape[1])
    for _ in range(num_iter):
        x = x + A.T @ (b - A @ x) / np.linalg.norm(A.T @ A, ord=2) - lmbda
        x = np.sign(x) * np.maximum(0, np.abs(x) - lmbda)
    return x

lmbda = 0.1 * np.linalg.norm(A.T @ b, ord=np.inf)

x_hat_PCD = PCD(A, b, lmbda)
x_hat_ISTA = ISTA(A, b, lmbda)

print("LASSO problem using PCD")
print(x_hat_PCD)

print("LASSO problem using ISTA")
print(x_hat_ISTA)

# Compute the error
error_PCD = np.linalg.norm(x - x_hat_PCD, ord=2)
error_ISTA = np.linalg.norm(x - x_hat_ISTA, ord=2)

print("Error using PCD: ", error_PCD)
print("Error using ISTA: ", error_ISTA)

#print number of 0's in x_hat_PCD and x_hat_ISTA
print("Number of 0's in x_hat_PCD: ", np.sum(x_hat_PCD == 0))
print("Number of 0's in x_hat_ISTA: ", np.sum(x_hat_ISTA == 0))

# The error is very high because the noise is very high. We can reduce the noise to get better results.




LASSO problem using PCD
[-0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.
 -0. -0. -0.  0. -0. -0. -0. -0.  0. -0.  0.  0. -0.  0. -0. -0.  0.  0.
 -0. -0.  0.  0.  0.  0. -0.  0. -0. -0.  0. -0.  0.  0.  0.  0.  0.  0.
  0.  0. -0. -0. -0. -0. -0.  0.  0.  0. -0. -0.  0. -0.  0.  0. -0.  0.
 -0. -0. -0. -0.  0. -0. -0.  0.  0.  0.  0.  0. -0.  0. -0. -0.  0. -0.
  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.
  0. -0.  0.  0.  0. -0.  0. -0.  0. -0.  0. -0.  0. -0.  0. -0. -0. -0.
 -0.  0. -0.  0.  0.  0.  0. -0.  0.  0. -0.  0. -0. -0.  0.  0.  0.  0.
 -0.  0.  0.  0. -0.  0. -0. -0. -0. -0. -0. -0. -0.  0. -0.  0. -0.  0.
  0.  0.  0. -0. -0. -0. -0.  0. -0.  0. -0.  0. -0.  0.  0.  0. -0. -0.
  0. -0.  0. -0.  0.  0. -0. -0.  0. -0.  0. -0. -0.  0.  0. -0.  0.  0.
  0.  0.]
LASSO problem using ISTA
[-0.         -0.         -0.18755552 -0.26623022 -0.09072483 -0.17841985
 -0.         -0.         -0.         -0.         -0.         -0.2